In [ ]:
import numpy as np
import tensorflow as tf
import gpflow
from gpflow.kernels import White, RBF
from gpflow.likelihoods import Gaussian
from deep_gp import DeepGP
import matplotlib.pyplot as plt

# Deep Gaussian Processes

-------------------

# Part 1 - Sampling From a Deep GP (worth 1 point)

#### You are provided with the following ingredients with which to build a GP model

In [ ]:
np.random.seed(111)
N = 500
x = np.linspace(-20, 20, num=N)

gp_var = 1.0  # GP variance
gp_len = 1.0  # GP lengthscale

def rbf_kernel(x1, x2):
    x1 = x1.reshape(-1, 1, 1)
    x2 = x2.reshape(1, -1, 1)
    r = np.sum(x1 - x2, axis=-1)
    return gp_var * np.exp(-0.5 * np.square(r) / np.square(gp_len))


def zero_mean(X):
    return np.zeros_like(X.reshape(-1, ))


def identity_mean(X):
    return X.reshape(-1, )

**Task 1a**: Generate a sample from a 3-layer deep GP with **zero mean** and an **RBF kernel**. Use the inputs, x, and parameters given above. Plot the sample path and the covariance matrix in each layer.

**Do you notice any strange behaviour in the sample path and/or covariance matrix? Write a short explanation of the behaviour and why you think it may be occuring.**

hint 1: you can use np.random.multivariate_normal() to sample from a Gaussian distribution. ***Note: you may need to add a small value to the diagonal of the covariance matrix to prevent numerical issues during sampling, e.g. + 1e-10 * np.eye(N)***

hint 2: use plt.imshow() to plot a covariance matrix

hint 3: always plot the initial inputs, x, on the x-axis of the sample plots.

**Task 1b**: Now generate a sample from a 3-layer deep GP, now with the **identity mean function for the first two layers**, and a **zero mean function for the final layer**. Plot the samples from each layer and the covariance matrices.

**Do you notice any differences in the behaviour between this sample and the previous one? Write a short explanation.**

-----------------------

# Part 2 - Training a Deep GP (worth 1 point)

Here we generate some training data using the step function

In [ ]:
tf.random.set_seed(0)
np.random.seed(123)
N = 40
X_train = np.linspace(-1, 1, N)[:, None]
f_step = lambda x: -1 if x < 0 else 1.
Y_train = np.reshape([f_step(x) for x in X_train], X_train.shape) + np.random.randn(*X_train.shape) * 1e-2
Xs = np.linspace(-2., 2, 300)[:, None]  # test inputs
plt.scatter(X_train, Y_train, color='k');

**Task 2a:** The ELBO calculation has been removed from the deep GP implementation. **Implement the ELBO yourself using the derivation on slide 16 of the lecture.**

A custom deep GP class has been created below. You are only required to finish the implementation of the ELBO, using the ingredients given in the code comments.

In [ ]:
class MyDeepGP(DeepGP):

    def elbo(self, data):
        """
        Implement the ELBO here using the derivation on slide 16 of the lecture notes
        :param data: Tuple of two tensors for input data X and labels Y.
        :return: Tensor representing ELBO.
        """
        X, Y = data  # mini-batch data for the current training step
        N_star = X.shape[0]  # number of points in the mini-batch
        N = self.num_data  # total number of data points
        S = self.num_samples  # the number of samples used to evaluate the marginals, q(f_{L,n})
        
        ############################################
        ######### Implement the ELBO here ##########
        ############################################
        
        # You will need the following ingredients:
        
        # self.predict_f() produces samples from the marginal, q(f_{L,n,s}):
        F_mean, F_var = self.predict_f(X, num_samples=S, full_cov=False)
        
        # now use self.likelihood.variational_expectations(F_mean, F_var, Y) to evaluate following:
        # \int q(f_{L,n,s}) ln p(y_n | f_{L,n,s}) d f_{L,n,s}

        # Compute the KL term. The KL for a single layer, i, can be computed as follows:
        # gpflow.kullback_leiblers.gauss_kl(self.layers[i].q_mu, self.layers[i].q_sqrt, K=self.layers[i].Ku)
        
        
        
        return ELBO

Now train the model and plot the posterior by running the following code (no modification needed):

In [ ]:
## Helper functions used to build and train the deep GP

def make_deep_GP(num_layers, X, Y, Z):
    kernels = []
    layer_sizes = []
    for l in range(num_layers):
        kernel = RBF(lengthscales=0.2, variance=1.0) + White(variance=1e-4)
        kernels.append(kernel)
        layer_sizes.append(1)

    dgp = MyDeepGP(X, Y, Z, kernels, layer_sizes, Gaussian(variance=1), num_samples=20)

    # init hidden layers to be near deterministic
    for layer in dgp.layers[:-1]:
        layer.q_sqrt.assign(layer.q_sqrt * 1e-2)
    return dgp


@tf.function()
def training_step(model, opt):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        obj = -model.elbo((X_train, Y_train))
        gradients = tape.gradient(obj, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return obj




## Now train the model and plot the result

M = 20  # num inducing points
num_layers = 3  # number of layers
Z_init = np.random.uniform(-1, 1, M)[:, None]
dgp = make_deep_GP(num_layers, X_train, Y_train, Z_init)
optimizer = tf.optimizers.Adam(learning_rate=0.01, epsilon=1e-08)

num_iters = 5000

for it in range(1, num_iters + 1):
    objective = training_step(dgp, optimizer)
    if np.mod(it, 100) == 0:
        print('iter %2d, ELBO: %1.4f' % (it, -objective.numpy()))

m_test, v_test = dgp.predict_y(Xs, num_samples=1000)
m_test, v_test = tf.reduce_mean(m_test, axis=0), tf.reduce_mean(v_test, axis=0)
lb = m_test[:, 0] - 1.96 * v_test[:, 0] ** 0.5
ub = m_test[:, 0] + 1.96 * v_test[:, 0] ** 0.5

plt.plot(Xs, m_test, color='b', alpha=0.3)
plt.fill_between(Xs[:, 0], lb, ub, color='b', alpha=0.05, label='95% confidence')
plt.xlim(min(Xs), max(Xs))

plt.title('Step Function: Deep GP')
plt.scatter(X_train, Y_train);

**Task 2b:** Plot the model output from each layer. **Write a brief explanation of what role you think each layer is playing in inference.**

In [ ]:
# Prediction in the deep GP is also done via sampling 
# The following code produces 50 predictive samples from each layer.
_, layers_means, layers_variances = dgp.predict_all_layers(Xs, num_samples=50, full_cov=False)
# You can get a good estimate of the posterior predictive mean and variance by taking the mean of the samples.

# Part 3: Model inference of lecture 9 (worth 2 points)

**Task 3**: Pick some small data set (or a subset of a big data set). Study a GP model of your own choosing, where the latent values can't be integrated out analytically. 

Examples (See Lecture 9):
- normal(mu,exp(eta)), mu ~ GP, eta ~ GP
  e.g. https://avehtari.github.io/casestudies/Motorcycle/motorcycle.html
- increasing magnitude model
  e.g. https://avehtari.github.io/casestudies/Birthdays/birthdays.html
- non-normal observation model, like Bernoulli, Poisson or negative-binomial
  e.g. http://avehtari.github.io/BDA_R_demos/demos_rstan/trafficdeaths.html

You may use any software you like (Stan, gpflow, gpytorch), and you may use full MCMC or
integrate over latent values using Laplace, EP or VI.

**Write a short report**:
1. describe your model and approach
2. what are optimized parameters vs. posterior of parameters
3. show predictive distribution with optimization vs. posterior predictive

For those familiar with GPflow, see
https://gpflow.readthedocs.io/en/develop/notebooks/advanced/mcmc.html#Example-3:-Fully-Bayesian-inference-for-generalized-GP-models-with-HMC

For those familiar with PyMC3, see
https://docs.pymc.io/notebooks/GP-Latent.html

**Hint**:
- Check the Lecture 9 video
- Studying the motorcycle casestudy is a good starting point
- If you have questions, check the course Slack channel https://join.slack.com/t/gp2021e4075/shared_invite/zt-l0x4rfc5-go1vZngUsvq9ix9wdMQAIQ